In [12]:
import cv2
import torch
import time
import torchvision.transforms as transforms
from PIL import Image

# Load the saved model and move it to the GPU
model = torch.load('C:/Users/Bruss/Desktop/Speciale/models/deployments/finalmodel_efficientnet_nolandmarks.pt')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Preprocessing of frame
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4429, 0.3043, 0.2806], 
                          std=[0.1187, 0.0874, 0.0728]),])

# Capture frame
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    if ret:
        start_time = time.time() # For Ms calculation
        # Convert the frame to RGB format
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Flip frame
        frame = cv2.flip(frame, 1)

        # Preprocess the frame
        pil_image = Image.fromarray(frame)
        image_tensor = preprocess(pil_image)
        image_tensor = image_tensor.unsqueeze(0)

        # Move the data to GPU or CPU
        image_tensor = image_tensor.to(device)

        # Pass the frame through the model
        with torch.no_grad():
            output = model(image_tensor)

        # Move the output to the CPU
        output = output.cpu()

        # Get the predicted class
        _, predicted = torch.max(output.data, 1)
        predicted_class = predicted.item()

        # Define the class names
        class_names = ['afraid',  'alone', 'boss', 'hello', 'tough']

        # Display the predicted class on the frame
        predicted_class_name = class_names[predicted_class]
        stop_time = time.time() # Stop timing for Ms calculation
        cv2.putText(frame, predicted_class_name, (250, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        cv2.putText(frame, "EfficientNet no landmarks", (10, 450), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
        cv2.putText(frame, f"{round((stop_time - start_time) * 1000,3)} ms", (460, 450), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

        # Show the frame
        cv2.imshow('Sign Language Recognition', frame)

    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
